<a href="https://colab.research.google.com/github/anko191/Python_Kaggle/blob/master/Tensorflow/For_gaining_scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 予測値をp_min ~ p_max の間に落とし込む

In [ ]:
p_min = 0.001
p_max = 0.999

In [ ]:
def metric(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, p_min, p_max)
    return -K.mean(y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred))

## BinaryCrossentropyのlabel_smoothingを使う
* import tensorflow.keras.losses as LOSS

In [ ]:
model.compile(optimizer = optimizer,
                loss = LOSS.BinaryCrossentropy(label_smoothing = 0.001),
                metrics = metric,
                )

## なんで？？
* ref - https://www.kaggle.com/c/lish-moa/discussion/185593
    * a

## MultilabelStratifiedShuffleSplit

In [ ]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold, MultilabelStratifiedShuffleSplit

In [ ]:
def MSKF(r_sub, res, n_splits = 4):
    for seed in seeds:
        for num , (train_idx, valid_idx) in tqdm(enumerate(MultilabelStratifiedShuffleSplit(n_splits = n_splits,
                                                                      random_state = seed,
                                                                      test_size = 0.2).split(train_target, train_target))):
            print(f'Multilabels - Fold : {num}')

            model = get_model(len(train.columns))
            # leakageするのでやめましょう

            checkpoint_path = f'now : {i}_Fold:{num}.hdf5'

            modelcheckpoint = ModelCheckpoint(checkpoint_path,
                                                      monitor='val_metric',
                                                      verbose = 0,
                                                      save_best_only=True,
                                                      save_weights_only = True,
                                                      mode = 'min', # 現象させろperiod = 1
                                                      )

            reduce_lr = ReduceLROnPlateau(monitor='val_metric',
                                          # lr = 0.1,
                                          factor=0.5,
                                          patience = 4,
                                          epsilon=1e-4,
                                          verbose = 1,
                                          mode = 'min')
            ES = keras.callbacks.EarlyStopping(monitor='val_metric',
                                          min_delta=0.00001,
                                          patience=8,
                                          verbose=2,
                                          mode='min')
            # print(train_idx) 横軸のindexかよ
            history = model.fit(train.values[train_idx],
                     train_target.values[train_idx],
                     validation_data = (train.values[valid_idx],
                                        train_target.values[valid_idx]),
                     epochs = 45,
                     batch_size = 128,
                     callbacks = [modelcheckpoint, reduce_lr, ES],
                     # verbose = 2,
                     )
            hists.append(history)
            
            
            model.load_weights(checkpoint_path)
            
            
            test_pred = model.predict(test.values)
            val_pred = model.predict(train.values[valid_idx])

            r_sub.loc[:,train_target.columns] += test_pred
            res.loc[valid_idx, train_target.columns] += val_pred
            print(' - - - - - - - - - (´・ω・｀) - - - - - - - -')
    r_sub.loc[:, train_target.columns] /= ((num+1) * n_seeds)
    res.loc[:, train_target.columns] /= n_seeds
    return r_sub, res